In [1]:
import pandas as pd

- Let's import our clean dataset

In [2]:
sent2 = pd.read_pickle('data/overview.pkl')

In [4]:
more = ['like','think','infj','feel','know','one','think','realli','thing','get','entp','intp','intj','would','entj','enfj','isfp','feel','istp','infp','enfp','esfj','estj','esfp','estp','istj','isfj','type','becaus','peopl','time']

def filter_tokens(string):
    return ([w for w in string.split() if not w in more])

In [6]:
sent2['posts2'] = sent2.posts.apply(lambda x: filter_tokens(x))

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.preprocessing import text
from tensorflow.keras import utils

train_size = int(len(sent2) * .7)
train_posts = sent2['posts2'][:train_size]
train_types = sent2['type'][:train_size]

test_posts = sent2['posts2'][train_size:]
test_types = sent2['type'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts)

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_types)

encoder = LabelEncoder()
encoder.fit(train_types)
y_train = encoder.transform(train_types)
y_test = encoder.transform(test_types)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [36]:
batch_size = 50
epochs = 15

# Build the model
model = Sequential()
model.add(Dense(1000, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 5464 samples, validate on 608 samples
Epoch 1/15
5464/5464 [==============================] - 5s 911us/sample - loss: 2.4204 - accuracy: 0.1887 - val_loss: 2.3356 - val_accuracy: 0.2615
Epoch 2/15
5464/5464 [==============================] - 4s 732us/sample - loss: 2.3115 - accuracy: 0.1989 - val_loss: 2.3135 - val_accuracy: 0.1875
Epoch 3/15
5464/5464 [==============================] - 4s 664us/sample - loss: 2.2637 - accuracy: 0.2191 - val_loss: 2.2682 - val_accuracy: 0.2829
Epoch 4/15
5464/5464 [==============================] - 4s 676us/sample - loss: 2.1916 - accuracy: 0.2520 - val_loss: 2.1541 - val_accuracy: 0.3076
Epoch 5/15
5464/5464 [==============================] - 4s 672us/sample - loss: 2.1142 - accuracy: 0.2818 - val_loss: 2.0975 - val_accuracy: 0.3240
Epoch 6/15
5464/5464 [==============================] - 4s 679us/sample - loss: 2.0543 - accuracy: 0.3000 - val_loss: 2.0618 - val_accuracy: 0.3207
Epoch 7/15
5464/5464 [==============================] - 4s 681us/

In [29]:
y_pred = model.predict_classes(x_test)

In [30]:
y_test2 =[]
for i in range(len(y_test)):
    y_test2.append(y_test[i].argmax())

In [31]:
from sklearn.metrics import accuracy_score,classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test2))
print(classification_report(y_test2, y_pred))
print(confusion_matrix(y_test2,y_pred))

accuracy 0.16327314636957357
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        54
           1       0.00      0.00      0.00       214
           2       0.00      0.00      0.00        77
           3       0.00      0.00      0.00       186
           4       0.00      0.00      0.00        11
           5       0.00      0.00      0.00        15
           6       0.00      0.00      0.00         9
           7       0.00      0.00      0.00        21
           8       0.16      1.00      0.28       425
           9       0.00      0.00      0.00       538
          10       0.00      0.00      0.00       327
          11       0.00      0.00      0.00       408
          12       0.00      0.00      0.00        53
          13       0.00      0.00      0.00        88
          14       0.00      0.00      0.00        69
          15       0.00      0.00      0.00       108

    accuracy                           0.16      26

In [8]:
mykey= {0:'ENFJ',1:'ENFP',2:'ENTJ',3:'ENTP',4:'ESFJ',5:'ESFP',6:'ESTJ',7:'ESTP',8:"INFJ",9: 'INFP',10:'INTJ',11:'INTP',12:'ISFJ',13:'ISFP',14:'ISTJ',15:'ISTP'}

In [5]:
me = "I think I've learned a lot of information in this class, I can do so much more than when I started this cohort but I know I'm still not at my best. I have a lot to learn and I will put the effort in to get there.that moment when you realize your model predicts the same thing over and over. not a good feeling"

In [37]:
np.median(model.predict_classes(tokenize.texts_to_matrix(me)))

10.0

In [29]:
notme = "that moment when you realize your model predicts the same thing over and over. not a good feeling"

np.median(model.predict_classes(tokenize.texts_to_matrix(notme)))

10.0

In [38]:
def PersonalityCheck(string):
    mykey= {0:'ENFJ',1:'ENFP',2:'ENTJ',3:'ENTP',4:'ESFJ',5:'ESFP',6:'ESTJ',7:'ESTP',
            8:"INFJ",9: 'INFP',10:'INTJ',11:'INTP',12:'ISFJ',13:'ISFP',14:'ISTJ',15:'ISTP'}
    pred = np.median(model.predict_classes(tokenize.texts_to_matrix(string)))
    return mykey[pred]

In [40]:
PersonalityCheck(me)

'INTJ'

In [43]:
# model.save("PersonalityPredictor.h5")

In [44]:
# pickle.dump(tokenize,open('tokenize.p','wb'))